In [1]:
from sklearn import preprocessing
import numpy as np
from pathlib import Path
import pandas as pd
import ipaddress

import sqlite3
import sys
import math
from random import randint, choice, uniform

In [ ]:
INPUT_FILE_TO_ENCODE = "/home/rhishi/Documents/tmp/fim_input_w_ts.csv"
#OUTPUT_PATTERN_DIR = "/Users/admin/Documents/codes/FIM/data/"
#INPUT_DIR_TO_DECODE= "/Users/admin/Downloads/labeled_output/Charm"

# Without Subnet and Time slice

In [2]:
def wo_subnet_timeshift(i_path,\
                       output_mapping_file ='/home/rhishi/Documents/tmp/mapping_wo_subnet_timeshift.txt',\
                       output_file_prefix = '/home/rhishi/Documents/tmp/wo_subnet'):
    df_in=pd.read_csv(i_path, sep=',', dtype=str)

    start_time = df_in['time'].astype(float).min() 

    df_in['time_diff'] = df_in['time'].astype(float) - start_time

    cols_to_delete = ['dir', 'label', 'time', 'time_diff']

    cols = df_in.columns.tolist()
    data_col  = cols.copy()

    for a_col in cols_to_delete:
        data_col.remove(a_col)

    for col in data_col:
        df_in[col] = df_in[col].apply(lambda x: col+'_'+str(x))

    uniq = []
    for col in data_col:
        uniq+=df_in[col].unique().tolist()

    le = preprocessing.LabelEncoder()
    le.fit(uniq)

    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    with open(output_mapping_file, 'w') as file:
        for k,v in le_name_mapping.items():
            file.write("{} {}\n".format(k,v))

    """    
    ################# with time step ######################
    STEP = 300.0
    ts = 0.0
    count = 0    
    while True:
        df_entries_t = df_in[ (df_in['time_diff'] > ts) &  (df_in['time_diff'] <= ts+STEP)]
        if df_entries_t.shape[0] > 0:
            for flow_dir in ['0','1']:

                df_entries = df_entries_t[df_entries_t['dir']==flow_dir][data_col]
                bfr = []
                for index,row in df_entries.iterrows():
                    entry = row.tolist()
                    bfr.append(le.transform(entry))

                df_w = pd.DataFrame(bfr)
                print('Writing {} {}'.format(flow_dir,ts))
                df_w.to_csv(output_file_prefix+"_dir_{}_step_{}.dat".format(flow_dir,count),\
                            header=False, index=False, mode='w', sep=' ')
            count+=1
        else:
            break

        ts += STEP
    """
        
    # all entries without ts
    for flow_dir in ['0','1']:
        bfr = []
        print('Direction: {}'.format(flow_dir))
        df_entries = df_in[df_in['dir']==flow_dir][data_col]
        for index,row in df_entries.iterrows():
            entry = row.tolist()
            bfr.append(le.transform(entry))

        df_w = pd.DataFrame(bfr)
        df_w.to_csv(output_file_prefix+"_dir_{}_wo_subnet_all.dat".format(flow_dir), index=False, mode='w', sep=' ')
        
    print('------------ Done ---------------------')

In [ ]:
STEP = 300.0
ts = STEP
while True:
    bfr = []
    df_entries_t = df_in[ df_in['time_diff'] >= ts]
        
    if df_entries_t.shape[0] > 0:
        for flow_dir in ['0','1']:
            
            df_entries = df_entries_t[df_entries_t['dir']==flow_dir][data_col]
        
            for index,row in df_entries.iterrows():
                entry = row.tolist()
                bfr.append(le.transform(entry))

            df_w = pd.DataFrame(bfr)
            print('Writing {} {}'.format(flow_dir,ts))
            df_w.to_csv("/Users/admin/Downloads/tmp/formatted_dir_{}_upto_{}.dat".format(flow_dir,ts),\
                        header=False, index=False, mode='w', sep=' ')
    else:
        break
        
    ts += STEP

In [ ]:
## Entries between timesteps

# Encoding

In [ ]:
def wo_subnet_all(i_path,\
                       output_mapping_file ='/home/rhishi/Documents/tmp/mapping_wo_subnet_timeshift.txt',\
                       output_file_prefix = '/home/rhishi/Documents/tmp/wo_subnet'):
for flow_dir in ['0','1']:
    bfr = []
    print('Direction: {}'.format(flow_dir))
    df_entries = df_in[df_in['dir']==flow_dir][data_col]
    for index,row in df_entries.iterrows():
        entry = row.tolist()
        bfr.append(le.transform(entry))
    
    df_w = pd.DataFrame(bfr)
    df_w.to_csv("/tmp/formatted_dir_{}.dat".format(flow_dir), index=False, mode='w', sep=' ')

# Decoding

In [ ]:
def fill_the_patterns(i_itemset, i_cols = ['iot', 'ext', 'sport', 'dport', 'sizeBin']):
    ret_patterns= []
    template_pattern = {}
    for pattern in i_itemset: #self._final_itemset[i_flow_dir]:
            
        template_pattern = {}
        for i in i_cols:
            template_pattern[i]='*'

        for value in pattern:
            val = value.split('_')
            if val[0] in i_cols:
                template_pattern[val[0]] = val[1]
                    
        #print(pattern, template_pattern)
        ret_patterns.append(template_pattern.values())

    return template_pattern.keys(),ret_patterns

def format_output(i_file):
    all_patterns= []
    with open(i_file, 'r') as rp:
        for line in rp.readlines():
            line = line.rstrip('\n')
            y = [int(i) for i in line.split(' ')[:-1]]
            pattern=list(le.inverse_transform(y))
            print(pattern)
            all_patterns.append(pattern)

        hdr, vals = fill_the_patterns(all_patterns)
        df_w = pd.DataFrame(vals, columns=hdr)
          
        file_to_write = i_file+'_labeled'
        df_w.to_csv(file_to_write, index=False, mode='w', sep=',') 

for filename in Path(INPUT_DIR_TO_DECODE).glob('*.txt'):
    print('Doing file {}'.format(filename))
    format_output(str(filename))

# With Subnet

In [ ]:
def get_masked_ip(i_ip, i_mask='16'):
    try:
        return str(ipaddress.IPv4Network(i_ip+"/"+i_mask, strict=False).network_address)
    except:
        return i_ip

def w_subnet():
    l_df=pd.read_csv(INPUT_FILE_TO_ENCODE, sep=',', dtype=str)

    l_df['iot'] = list(map(get_masked_ip, l_df['iot']))
    l_df['ext'] = list(map(get_masked_ip, l_df['ext']))

    cols = l_df.columns.tolist()
    data_col  = cols.copy()
    data_col.remove('dir')
    data_col.remove('label')          
    for col in data_col:
        l_df[col] = l_df[col].apply(lambda x: col+'_'+str(x))

    uniq = []
    for col in data_col:
        uniq+=l_df[col].unique().tolist()

    le_s = preprocessing.LabelEncoder()
    le_s.fit(uniq)

    le_name_mapping = dict(zip(le_s.classes_, le_s.transform(le_s.classes_)))

    with open('/Users/admin/Downloads/mapping_w_subnet16s.txt', 'w') as file:
        for k,v in le_name_mapping.items():
            file.write("{} {}\n".format(k,v))

    for flow_dir in ['0','1']:
        bfr = []
        print('Direction: {}'.format(flow_dir))
        df_entries = l_df[l_df['dir']==flow_dir][data_col]
        for index,row in df_entries.iterrows():
            entry = row.tolist()
            bfr.append(le_s.transform(entry))

        df_w = pd.DataFrame(bfr)
        df_w.to_csv("/Users/admin/Downloads/subnet16_dir_{}.dat".format(flow_dir), index=False, mode='w', sep=' ')

#  With Subnet and Time slice

In [3]:
def get_masked_ip(i_ip, i_mask='16'):
    try:
        return str(ipaddress.IPv4Network(i_ip+"/"+i_mask, strict=False).network_address)
    except:
        return i_ip

def w_subnet_timeshift(i_path,\
                       output_mapping_file ='/home/rhishi/Documents/tmp/mapping_w_subnet_timeshift.txt',\
                       output_file_prefix = '/home/rhishi/Documents/tmp/w_subnet'):
    l_df=pd.read_csv(i_path, sep=',', dtype=str)

    l_df['iot'] = list(map(get_masked_ip, l_df['iot']))
    l_df['ext'] = list(map(get_masked_ip, l_df['ext']))

    start_time = l_df['time'].astype(float).min() 

    l_df['time_diff'] = l_df['time'].astype(float) - start_time

    cols_to_delete = ['dir', 'label', 'time', 'time_diff']

    cols = l_df.columns.tolist()
    data_col  = cols.copy()
    for a_col in cols_to_delete:
        data_col.remove(a_col)

    for col in data_col:
        l_df[col] = l_df[col].apply(lambda x: col+'_'+str(x))

    uniq = []
    for col in data_col:
        uniq+=l_df[col].unique().tolist()

    le_s = preprocessing.LabelEncoder()
    le_s.fit(uniq)

    le_name_mapping = dict(zip(le_s.classes_, le_s.transform(le_s.classes_)))

    with open(output_mapping_file, 'w') as file:
        for k,v in le_name_mapping.items():
            file.write("{} {}\n".format(k,v))

    """
    # with time step
    
    STEP = 300.0
    ts = 0.0
    count = 0
    while True:
        df_entries_t = l_df[ (l_df['time_diff'] > ts) &  (l_df['time_diff'] <= ts+STEP)]

        if df_entries_t.shape[0] > 0:
            for flow_dir in ['0','1']:

                df_entries = df_entries_t[df_entries_t['dir']==flow_dir][data_col]
                bfr = []
                for index,row in df_entries.iterrows():
                    entry = row.tolist()
                    bfr.append(le_s.transform(entry))

                df_w = pd.DataFrame(bfr)
                print('Writing {} {}'.format(flow_dir,ts))
                df_w.to_csv(output_file_prefix+"_dir_{}_step_{}.dat".format(flow_dir,count),\
                            header=False, index=False, mode='w', sep=' ')
            count+=1
        else:
            break

        ts += STEP
    """   
    # all entries without time slice
    for flow_dir in ['0','1']:
        bfr = []
        print('Direction: {}'.format(flow_dir))
        df_entries = l_df[l_df['dir']==flow_dir][data_col]
        for index,row in df_entries.iterrows():
            entry = row.tolist()
            bfr.append(le_s.transform(entry))

        df_w = pd.DataFrame(bfr)
        df_w.to_csv(output_file_prefix+"_dir_{}_w_subnet_all.dat".format(flow_dir), index=False, mode='w', sep=' ')
        
    print('------------ Done ---------------------')

In [ ]:
def get_bin(i_size):
    if i_size <= 100:
        return 'S'
    elif i_size > 100 and i_size <=500:
        return 'M'
    else:
        return 'L'
    
def get_all_entries(i_subsampling = True):
    con = sqlite3.connect("/home/rhishi/workspace/nus/botnet_sim/manager/run_2/sim.db")
    crsr = con.cursor()
    ret_list = []
    event_count = {}

    # subsample login and scan outs
    
    if i_subsampling:
        login_attempts = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where pcount> 0 and dir = '1' ORDER BY RANDOM() LIMIT {}".format(randint(2000,3000))
        scan_outs = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '22' and dir = '0' ORDER BY RANDOM() LIMIT {}".format(randint(2000,3000))
    else:    
        login_attempts = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where pcount> 0 and dir = '1'"
        scan_outs = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '22' and dir = '0'"

    scan_in = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where pcount = 0 and dir = '1' and extIP not like '10.2.%'" 
    loader = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '8000' and dir = '0'"
    cnc = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '4567' and dir = '0'"
    http_attack = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '80' and dir = '0'"
    dns_attack = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount, time from sim where dport = '53' and dir = '0'"

    for q,l in zip([login_attempts,scan_outs,scan_in,loader,cnc,http_attack,dns_attack], ['login', 'scan-out', 'scan-in', 'loader', 'cnc', 'http_ddos', 'dns_rddos']): 

      crsr.execute(q)
      l_op = crsr.fetchall()
      if not l_op is None:
            event_count[l] = len(l_op)
            for entry in l_op:
                try:
                    insize = entry[6]
                    outsize = entry[7]
                    if entry[6] is None:
                        insize = 0
                    if entry[7] is None:
                        outsize = 0
                    size = int(insize+outsize)
                    size_bin = get_bin(size)
                    ret_list.append([entry[4], entry[0], entry[1], entry[2]+'.'+ entry[5],\
                        entry[2]+'.'+ entry[3], size_bin, entry[8], l])
                except KeyError:
                    print('Unknown direction entries in db.')
    print(len(ret_list))
    df_w = pd.DataFrame(ret_list)
    df_w.to_csv('/home/rhishi/Documents/tmp/entries_reduced.csv', header=['dir', 'iot', 'ext', 'sport', 'dport', 'sizeBin', 'time', 'label'],index=False)
    print(event_count)
    con.close()


get_all_entries(i_subsampling = True)

In [ ]:
def get_ip_address(i_network):
    net = ipaddress.IPv4Network(i_network)
    return net[randint(2, net.num_addresses-2)].exploded


def gen_noises(i_limit, output_file_name):
    con = sqlite3.connect("/home/rhishi/workspace/nus/botnet_sim/manager/run_2/sim.db")
    crsr = con.cursor()
    final_list = []
    
    LIMIT = randint(i_limit, i_limit+1000)
    
    
    try:
        # Add logic to add noise afterwards. 
        # get all the compromised iots.
        iots = []
        ports = [22,23,2323,443,80,8080]
        crsr.execute('select distinct intIP from sim where dir ="1"')
        l_op = crsr.fetchall()
        for i in l_op :
            iots.append(i[0])

        crsr.execute("select min(time), max(time) from sim")
        min_time, max_time = crsr.fetchone()
        print('Adding extra noise ......................')

        while True:
          iot = choice(iots)
          ext = get_ip_address("0.0.0.0/0")
          proto = choice([6,17])
          #dport = choice(ports)
          dport = randint(20,1000)
          sport = randint(11000,65000)
          ts = uniform(float(min_time), float(max_time))
          item_list = [iot, ext, proto, dport, 1, sport, 1, 1, 40, 40, 1, 0, ts]
          where_clause_prefix = "select incount from sim "
          where_clause = "where "
          for i,j in enumerate(['intIP', 'extIP', 'proto', 'dport', 'dir']):
            where_clause+= '{} = "{}" AND '.format(j, item_list[i])

          #print(where_clause_prefix + where_clause[:-5])
          crsr.execute(where_clause_prefix + where_clause[:-5])
          l_op = crsr.fetchone()
          
          if l_op is None:
            l_pattern = "1,{},{},6.{},6.{},S,{},scan-in-noise\n".format(iot, ext, sport, dport, ts)
            if not l_pattern in final_list: 
              #final_list.append(",".join(str(e) for e in item_list)+"\n")
              final_list.append(l_pattern)

          if len(final_list) >= LIMIT:
            break


        print('Adding legit traffic ......................')

        LIMIT1 = randint(30,50)
        iots = []
        crsr.execute('select distinct intIP from sim where dir = "0" and dport = "443"')
        l_op = crsr.fetchall()
        for i in l_op :
            iots.append(i[0])

        while True:
          iot = choice(iots)
          sport = randint(11000,65000)
          ext = "www.random.org."
          dport = "443"
          ts = uniform(min_time, max_time)
          item_list = [iot, ext, 6, dport, 0, sport, 1, 1, 40, 40, 1, 0, ts]
          where_clause_prefix = "select incount from sim "
          where_clause = "where "
          for i,j in enumerate(['intIP', 'extIP', 'proto', 'dport', 'dir', 'sport']):
            where_clause+= '{} = "{}" AND '.format(j, item_list[i])

          #print(where_clause_prefix + where_clause[:-5])
          crsr.execute(where_clause_prefix + where_clause[:-5])
          l_op = crsr.fetchone()
          if l_op is None:
           #final_list.append(",".join(str(e) for e in item_list)+"\n")
           final_list.append("0,{},{},6.{},6.{},L,{},legit\n".format(iot, ext, sport, dport, ts))

          if len(final_list) >= LIMIT+LIMIT1:
            break

    except:
      print('Exception.....')
      print(sys.exc_info())

    con.close()

    with open(output_file_name, 'w') as fp:
        fp.writelines(final_list)

In [ ]:
total_entries = 7866
noise_muliple = 1
gen_noises(total_entries*noise_muliple, "/home/rhishi/Documents/tmp/extra_noise_1_w_ts.txt")
noise_muliple = 2
gen_noises(total_entries*noise_muliple, "/home/rhishi/Documents/tmp/extra_noise_2_w_ts.txt")
noise_muliple = 4
gen_noises(total_entries*noise_muliple, "/home/rhishi/Documents/tmp/extra_noise_3_w_ts.txt")
noise_muliple = 8
gen_noises(total_entries*noise_muliple, "/home/rhishi/Documents/tmp/extra_noise_4_w_ts.txt")

In [ ]:
w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_1_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_1/mapping_noise_1_w_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_1/input"
                )

w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_2_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_2/mapping_noise_2_w_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_2/input"
                )

w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_3_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_3/mapping_noise_3_w_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_3/input",
                )

w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_4_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_4/mapping_noise_4_w_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_4/input",
                )


In [ ]:
wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_1_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_1/mapping_noise_1_wo_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_1/input"
                )

wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_2_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_2/mapping_noise_2_wo_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_2/input"
                )

wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_3_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_3/mapping_noise_3_wo_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_3/input",
                )

wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_4_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_4/mapping_noise_4_wo_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_4/input",
                )

In [ ]:
w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_0_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_0/mapping_noise_0_w_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_0/input")

wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_0_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_0/mapping_noise_0_wo_subnet_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_0/input"
                )

In [ ]:
w_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_no_cnc_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_1/mapping_noise_1_w_subnet_no_cnc_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/with_subnet/noise_1/input")

wo_subnet_timeshift("/home/rhishi/Documents/tmp/fim_input_no_cnc_w_ts.csv",
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_1/mapping_noise_1_wo_subnet_no_cnc_timeshift.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/no_subnet/noise_1/input"
                )

In [8]:
w_subnet_timeshift('/home/rhishi/Documents/tmp/submission/output/kushans_way/UNSW_attack_transactions.csv',
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/output/kushans_way/mapping_UNSW_w_subnet.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/output/kushans_way/input")

wo_subnet_timeshift('/home/rhishi/Documents/tmp/submission/output/kushans_way/UNSW_attack_transactions.csv',
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/output/kushans_way/mapping_UNSW_wo_subnet.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/output/kushans_way/input")

Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------


In [5]:
for noise in range(0,5):
    print("Noise level -> {}".format(noise))
    for gateway in range(1,8):
        w_subnet_timeshift('/tmp/gateway_wise/gateway_{}/gateway_{}_noise_{}'.format(gateway, gateway, noise),
                   output_mapping_file = "/tmp/gateway_wise/gateway_{}/mapping_gw_{}_noise_{}_w_subnet.txt".format(gateway, gateway, noise),
                   output_file_prefix = "/tmp/gateway_wise/gateway_{}/input_gw_{}_noise_{}".format(gateway, gateway, noise))

        wo_subnet_timeshift('/tmp/gateway_wise/gateway_{}/gateway_{}_noise_{}'.format(gateway, gateway, noise),
                   output_mapping_file = "/tmp/gateway_wise/gateway_{}/mapping_gw_{}_noise_{}_wo_subnet.txt".format(gateway, gateway, noise),
                   output_file_prefix = "/tmp/gateway_wise/gateway_{}/input_gw_{}_noise_{}".format(gateway, gateway, noise))

Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction: 0
Direction: 1
------------ Done ---------------------
Direction:

In [19]:
def get_masked_ip(i_ip, i_mask='16'):
    try:
        return str(ipaddress.IPv4Network(i_ip+"/"+i_mask, strict=False).network_address)
    except:
        return i_ip


def timeshift_dataset_no_encode(i_path, i_subnet, output_file_prefix):
    df_in=pd.read_csv(i_path, sep=',', dtype=str)
    
    if i_subnet == True:
        print('With subnetting.')
        df_in['iot'] = list(map(get_masked_ip, df_in['iot']))
        df_in['ext'] = list(map(get_masked_ip, df_in['ext']))
    else:
        print('Without subnetting.')
    
    start_time = df_in['time'].astype(float).min() 

    df_in['time_diff'] = df_in['time'].astype(float) - start_time

    cols_to_delete = ['dir', 'time', 'time_diff']

    cols = df_in.columns.tolist()
    data_col  = cols.copy()

    for a_col in cols_to_delete:
        data_col.remove(a_col)
    """
    for col in data_col:
        df_in[col] = df_in[col].apply(lambda x: col+'_'+str(x))
    """

    uniq = []
    for col in data_col:
        uniq+=df_in[col].unique().tolist()

    ################# with time step ######################
    STEP = 300.0
    ts = 0.0
    count = 0    
    while True:
        df_entries_t = df_in[ (df_in['time_diff'] > ts) &  (df_in['time_diff'] <= ts+STEP)]
        if df_entries_t.shape[0] > 0:
                df_entries = df_entries_t[data_col]
                """
                bfr = []
                for index,row in df_entries.iterrows():
                    entry = row.tolist()
                    bfr.append(entry)

                df_w = pd.DataFrame(bfr)
                print('Writing {} {}'.format(flow_dir,ts))
                """
                df_entries.to_csv(output_file_prefix+"_dir_step_{}.dat".format(count),\
                            header=False, index=False, mode='w', sep=' ')
                count+=1
        else:
            break

        ts += STEP
        
    print('------------ Done ---------------------')

In [20]:
timeshift_dataset_no_encode('/home/rhishi/Documents/tmp/fim_input_0_w_ts.csv', i_subnet=False,\
                            output_file_prefix='/tmp/timestampwise_wo_subnet')
timeshift_dataset_no_encode('/home/rhishi/Documents/tmp/fim_input_0_w_ts.csv', i_subnet=True,\
                            output_file_prefix='/tmp/timestampwise_w_subnet')

Without subnetting.
------------ Done ---------------------
With subnetting.
------------ Done ---------------------


In [3]:
wo_subnet_timeshift('/home/rhishi/Documents/tmp/paper_example.csv',
                   output_mapping_file = "/home/rhishi/Documents/tmp/submission/output/paper_mapping_wo_subnet.txt",
                   output_file_prefix = "/home/rhishi/Documents/tmp/submission/output/paper_input")

Direction: 0
Direction: 1
------------ Done ---------------------
